# Demos: Lecture 8

In [1]:
import pennylane as qml
from pennylane import numpy as np
from lecture08_helpers import compute_accuracy
from lecture08_helpers import *
import matplotlib.pyplot as plt

# Loads the training and testing data for you
train_X = np.load("data/l8_train_X.npy")
train_y = np.load("data/l8_train_y.npy")
test_X = np.load("data/l8_test_X.npy")
test_y = np.load("data/l8_test_y.npy")

train_X = np.array(train_X, requires_grad=False)
train_y = np.array(train_y, requires_grad=False)
test_X = np.array(test_X, requires_grad=False)
test_y = np.array(test_y, requires_grad=False)

In [2]:
np.set_printoptions(suppress=True)

In [3]:
test_X

tensor([[-0.07161386, -0.22872007,  0.88470143, -0.10581928],
        [ 1.64567103, -0.66481804, -0.57328593,  0.07178712],
        [ 0.26837931,  0.42790794,  0.15068389, -0.45816579],
        [ 0.23364672,  0.32846986,  0.6326616 , -0.09151491],
        [-1.05684875, -1.42349957, -1.3374081 ,  1.93054905],
        [-0.64311332, -2.00718505, -1.60691971,  1.43026965],
        [ 0.13988981,  0.28099617,  0.50405069, -0.44432248],
        [ 0.05787516,  0.09546232,  1.28959618, -0.14956287],
        [-0.42451288,  0.00777202,  0.83388789, -0.23143627],
        [ 2.15115809,  0.02331367, -0.83677661,  0.59475188],
        [ 2.38297868, -1.03423083, -0.66092546,  0.68480826],
        [ 1.95689934, -0.60490362, -0.8247449 , -0.00755088],
        [-0.51139536, -0.1408783 , -0.29199464, -0.09000482],
        [-0.61007558, -2.01992869, -1.49499303,  1.90725233],
        [ 1.88651323, -0.70172563, -0.54178926,  0.28770913],
        [ 0.47585128,  0.4080172 ,  0.01257563, -0.14044039],
        

## Hands-on


Design your own VQC to classify this data! Try to beat my record of 93.3\% accuracy.

In [4]:
NUM_WIRES = 4
NUM_PARAMS = 3
NUM_LAYERS = 2

In [5]:
weights = np.random.normal(size=(NUM_WIRES*NUM_LAYERS, NUM_PARAMS))

In [6]:
weights

tensor([[-1.51912792,  1.14954625, -0.18405269],
        [ 0.98275933, -0.74081779, -1.32335415],
        [ 1.31473985, -0.17146874, -1.38557605],
        [ 0.38983346,  0.39128117,  1.93891118],
        [-0.0716272 , -0.9895772 , -1.23892093],
        [-1.7189187 ,  0.57680439,  1.96789488],
        [ 1.33150642, -1.05369049,  0.90675358],
        [-0.99039184, -0.41692748, -0.37955812]], requires_grad=True)

In [7]:
weights[0]

tensor([-1.51912792,  1.14954625, -0.18405269], requires_grad=True)

In [8]:
dev = qml.device('default.qubit', wires=NUM_WIRES)

@qml.qnode(dev)
def vqc_model(data, weights):
    # YOUR CODE HERE
    # Design an ansatz for this data: choose how the data is encoded
    # and the gates that the weights are input into.
    qml.AngleEmbedding(data, wires=[0,1,2,3])
    qml.Rot(*weights[0], wires=0)
    qml.Rot(*weights[1], wires=1)
    qml.Rot(*weights[2], wires=2)
    qml.Rot(*weights[3], wires=3)

    qml.SWAP(wires=[0,1])
    qml.SWAP(wires=[1,2])
    qml.SWAP(wires=[2,3])

    qml.Rot(*weights[4], wires=0)
    qml.Rot(*weights[5], wires=1)
    qml.Rot(*weights[6], wires=2)
    qml.Rot(*weights[7], wires=3)

    qml.SWAP(wires=[0,1])
    qml.SWAP(wires=[1,2])
    qml.SWAP(wires=[2,3])

    return qml.expval(qml.PauliX(0))

In [9]:
print(qml.draw(vqc_model, expansion_strategy="device")(train_X[0], weights))

0: ──RX(-0.77)──Rot(-1.52,1.15,-0.18)─╭SWAP──Rot(-0.07,-0.99,-1.24)──────────────────────
1: ──RX(-1.89)──Rot(0.98,-0.74,-1.32)─╰SWAP─╭SWAP────────────────────Rot(-1.72,0.58,1.97)
2: ──RX(-1.03)──Rot(1.31,-0.17,-1.39)───────╰SWAP───────────────────╭SWAP────────────────
3: ──RX(1.47)───Rot(0.39,0.39,1.94)─────────────────────────────────╰SWAP────────────────

──╭SWAP───────────────────────────────┤  <X>
──╰SWAP───────────────────╭SWAP───────┤     
───Rot(1.33,-1.05,0.91)───╰SWAP─╭SWAP─┤     
───Rot(-0.99,-0.42,-0.38)───────╰SWAP─┤     


In [10]:
def loss(weights):
    # YOUR CODE HERE
    # Design a new loss function, or adapt the helper function

    loss_sum = 0.0

    for idx in range(train_X.shape[0]):
        point = train_X[idx]
        true_expval = train_y[idx]

        estimated_expval = vqc_model(point, weights)
        loss_sum += (estimated_expval - true_expval) ** 2

    return loss_sum / train_X.shape[0]

In [11]:
# Set up initial parameters, and write an optimization loop to train your model.

opt = qml.GradientDescentOptimizer(stepsize=0.1)

n_its = 100

loss_track = []

for it in range(n_its):
    weights, _loss = opt.step_and_cost(loss, weights)
    if it % 5 == 0:
        our_preds = make_predictions(train_X, vqc_model, weights)
        print(f"Loss at iteration {it} = {_loss}  Accuracy = {compute_accuracy(our_preds, train_y)}")
    loss_track.append(_loss)

Loss at iteration 0 = 2.140714062539886  Accuracy = 0.0
Loss at iteration 5 = 1.4635276737930143  Accuracy = 0.06666666666666667
Loss at iteration 10 = 1.0473916009196038  Accuracy = 0.24666666666666667
Loss at iteration 15 = 0.7233030445051635  Accuracy = 0.31333333333333335
Loss at iteration 20 = 0.6398248178662778  Accuracy = 0.32
Loss at iteration 25 = 0.6266502862132027  Accuracy = 0.32
Loss at iteration 30 = 0.6242421718830921  Accuracy = 0.32666666666666666
Loss at iteration 35 = 0.6236844533174594  Accuracy = 0.32666666666666666
Loss at iteration 40 = 0.6235411916933492  Accuracy = 0.32666666666666666
Loss at iteration 45 = 0.6235032180580469  Accuracy = 0.32666666666666666
Loss at iteration 50 = 0.6234930565868112  Accuracy = 0.32666666666666666
Loss at iteration 55 = 0.6234903283654939  Accuracy = 0.32666666666666666
Loss at iteration 60 = 0.6234895948610173  Accuracy = 0.32666666666666666
Loss at iteration 65 = 0.6234893975260184  Accuracy = 0.32666666666666666
Loss at itera

In [12]:
# def make_predictions(data, model, weights):
#     # YOUR CODE HERE
#     # Modify the make_predictions function based on how you set up your classifier.
#
#     return

After training, you can use the code below to evaluate the metrics.

In [13]:
my_training_predictions = make_predictions(train_X, vqc_model, weights)
my_testing_predictions = make_predictions(test_X, vqc_model, weights)
                                          
training_accuracy = compute_accuracy(my_training_predictions, train_y)
testing_accuracy = compute_accuracy(my_testing_predictions, test_y)

print(f"Training accuracy = {training_accuracy}")
print(f"Testing accuracy = {testing_accuracy}")

Training accuracy = 0.32666666666666666
Testing accuracy = 0.3333333333333333
